In [1]:
!pip install tensorflow-gpu
!pip install deeppavlov
!python -m deeppavlov install squad_bert
!pip install fasttext
!pip install transformers==2.6.0
!python -m deeppavlov install bert_sentence_embedder

2021-05-27 15:14:12.901 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'squad_bert' as '/usr/local/lib/python3.7/dist-packages/deeppavlov/configs/squad/squad_bert.json'
  Cloning https://github.com/deepmipt/bert.git (to revision feat/multi_gpu) to /tmp/pip-req-build-ei6vu4ll
  Running command git clone -q https://github.com/deepmipt/bert.git /tmp/pip-req-build-ei6vu4ll
  Created wheel for bert-dp: filename=bert_dp-1.0-cp37-none-any.whl size=23581 sha256=34cfc07f217c1f5e287fc2205bb7fffd577ebe4df2ca2d2551c4dcf68c79b2a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-jmojjbsx/wheels/1e/41/94/886107eaf932532594886fd8bfc9cb9d4db632e94add49d326
Successfully built bert-dp
2021-05-27 15:14:17.824 WARNING in 'deeppavlov.utils.pip_wrapper.pip_wrapper'['pip_wrapper'] at line 34: found tensorflow-gpu installed, so upgrading it instead of tensorflow
2021-05-27 15:14:26.578 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'bert_sentence_embedder' as '/

# Лемматизатор

In [1]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

class Lemmatizer():
  def __init__(self):
    self.morph = pymorphy2.MorphAnalyzer()
  
  def process(self, tokens):
    lemmatized = []

    for token in tokens:
      lemmatized.append(self.morph.parse(token)[0].normal_form)

    return lemmatized

  def show_diff(self, tokens):
    lemmatized = self.process(tokens)

    diff_lemm = pd.DataFrame({
        'original': filtered_tokens,
        'w/lemm': lemmatized,
    }, columns=['original', 'w/lemm'])

    return diff_lemm

# Стеммер

In [2]:
from nltk.stem import SnowballStemmer

class Stemmer():
  def __init__(self):
    self.snowball = SnowballStemmer(language="russian")

  def process(self, tokens):
    stemmed = []

    for token in tokens:
      stemmed.append(snowball.stem(token))

    return stemmed
  
  def show_diff(self, tokens):
    stemmed = self.process(tokens)

    diff_stemming = pd.DataFrame({
        'original': tokens,
        'w/stem': stemmed,
    }, columns=['original', 'with stemming'])

    return diff_stemming

# Предобработчик

In [35]:
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

class Preprocessor():
  def __init__(self):
    self.stopwords = stopwords.words("russian")
    self.stemmer = Stemmer()
    self.lemmatizer = Lemmatizer()

  def remove_digits(self, text):
    return re.sub("[0-9]+", " ", str(text))

  def to_lower(self, text):
    return text.lower()

  def remove_stopwords(self, tokens):
      filtered_tokens = []

      for token in tokens:
          if token not in stop_words:
              filtered_tokens.append(token)

      return filtered_tokens

  def remove_punctuation(self, text):
    return re.sub(r'(['+string.punctuation+'])', ' ', text)

  def remove_tabs(self, text):
    text = text.replace("\\n", " ")
    text = text.replace("\\t", " ")
    text = text.replace("\\xa0", " ")
    text = text.replace("\\xc2", " ")

    return text

  def lemmatize(self, tokens):
    return self.lemmatizer.process(tokens)
  
  def stemm(self, tokens):
    return self.stemmer.process(tokens)
  
  def main_pipeline(self, text):
    text = self.remove_digits(text)
    text = self.to_lower(text)
    text = self.remove_punctuation(text)
    text = self.remove_tabs(text)

    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Классификатор

In [39]:
from deeppavlov import build_model
import torch

class Classifier():
  def __init__(self, path='/content/drive/MyDrive/BERT/configs/bert/rubert_sentiment.json'):
    self.model = build_model(path)
    self.sentiment_map = {"0": "NEUTRAL", "1": "POSITIVE", "2": "NEGATIVE"}

  def convert_id_to_sentiment(self, id):
    if id not in self.sentiment_map:
      raise TypeError
    
    return self.sentiment_map[id]

  @torch.no_grad()
  def predict(self, inputs, return_class=True):
    predicted = self.model([inputs])
    if return_class:
      predicted = self.convert_id_to_sentiment(predicted[0])
    return predicted

In [45]:
text = [
        "Я тебя люблю", 
        "Ненавиж всех вокруг.", 
        "Какой хороший день",
        "Сегодня хорошая погода", 
        "Я радовался встрече с тобой", 
        "Мне нравится эта музыкальная композиция",
        "В больнице была ужасная очередь", 
        "Сосед с верхнего этажа мешает спать", 
        "Маленькая девочка потерялась в торговом центре",
        "На первый взгляд в квартире происходила борьба между мужчиной и женщиной. Во всяком случае, расположение тел об этом говорило. Но опытных сыщиков, которые прибыли на место преступления, насторожили некоторые детали, из чего они сделали вывод, что к убийству причастен кто-то посторонний.",
        "Убил, виновен, понесет заслуженное наказание. Но почему журналисты делают такой акцент на компьютерной игре, в которой он насмотрелся сцен насилия, а не на дозе алкоголя, несовместимой с нормальной жизнедеятельностью организма?",
        "Обслуживаю оргтехнику и компы в офисах. Пришел по заявке - не печатает. Отвалилась сетевая печать. Замучал весь системник где установлен принтер, - ставлю сетевой доступ - ОК, захожу смотрю, снова нет доступа. Сносил все настройки сети, не помогло. Начал переустановки системы, заявка из соседнего кабинета - не печатает. Причем только этот комп не может подключиться к удалённому принтеру, остальные могут. Стал играть в детектива, что, как, когда - оказалось компьютер обновился и после этого всё пропало. Пришлось удалить последнее обновление КВ 5003137 и сразу всё заработало, на следующий день ещё на 3 компьютерах такая же фигня. Удаляю эту обнову, пытаюсь отключить автообновление, но всё равно это не правильно, а вот что сделать чтоб правильно. Win 10 2004. Тру админы помогите!",
        "Всем добрый день, вечер, ночь, утро! Просто хочу поделиться радостью - сегодня мой сын первый день на практике в театре.В общем-то ничего необычного, он не будущий актер, а скажем так - техсостав. Но! Ровно 15 лет назад мы проходили консультацию в 6-й детской психиатрической больнице и  научный консультант  сказала мне буквально следующее - Готовьтесь мамочка к тому, что ваш ребенок никогда не будет говорить-писать -читать и тд.Порыдала, пострадала и мы начали бороться за светлое будущее... думаю, что мы добились многого и сегодня  я счастлива! Не сдавайтесь!",
        "Всем привет. Сегодня впервые в жизни я плакала от счастья. У моей маленькой внучки не подтвердились подозрения на опухоль мозга. Месяц ожидания и надежды вперемешку с отчаянным страхом. Вот оно - счастье.",
        "Однако присяжные единодушно вынесли вердикт: «Виновен, снисхождению не подлежит». И вот — судья краевого суда вынес свое решение: пожизненное лишение свободы с отбыванием в колонии строгого режима. Согласно Уголовному кодексу, за такое преступление предусмотрена и высшая мера наказания, но в России смертная казнь не проводится.",
        "Терапевтическое отделение Не дай бог Вам попасть в терапевтическое отделение №3. Отношение просто свинское, что лечащего врача (Альфия Хасяновна), что зав. отделением, г-жи Кошкиной Г. И. Сын попал в это отделение с сильными головными болями, скачущим пульсом и давлением. Кололи витамины, назначили массаж спины еще что-то, потом лечащий врач назначил физиотерапию на больной сустав (внутри собранный на пластины), поднялась температура, которая держалась 4 дня, болел сустав, отдавала по всей ноге. На мои просьбы вызвать хирурга, ответа не следовало. Все же через 4 дня соизволили посмотреть, ничего не поняли. На наши вопросы в ответ услышали, что вы тут вообще у нас делаете, мы не должны вас лечить, это не наш профиль. Мой совет - НЕ ПОПАДАЙТЕ СЮДА, лучше в любую другую больницу, чем сюда. ",

        ]

In [36]:
prep = Preprocessor()

In [41]:
classifier = Classifier()

2021-05-27 18:48:23.967 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/drive/MyDrive/BERT/sentiment_bert_model/classes.dict]
2021-05-27 18:48:24.103 INFO in 'deeppavlov.models.torch_bert.torch_transformers_classifier'['torch_transformers_classifier'] at line 178: From pretrained /content/drive/MyDrive/BERT/rubert-base-cased-sentiment/.
2021-05-27 18:48:30.733 INFO in 'deeppavlov.models.torch_bert.torch_transformers_classifier'['torch_transformers_classifier'] at line 203: Load path /content/drive/MyDrive/BERT/sentiment_bert_model/model is given.
2021-05-27 18:48:30.737 INFO in 'deeppavlov.models.torch_bert.torch_transformers_classifier'['torch_transformers_classifier'] at line 210: Load path /content/drive/MyDrive/BERT/sentiment_bert_model/model.pth.tar exists.
2021-05-27 18:48:30.738 INFO in 'deeppavlov.models.torch_bert.torch_transformers_classifier'['torch_transformers_classifier'] at line 211: Initializing `TorchTransforme

In [46]:
for sent in text:
  prepared_text = prep.main_pipeline(sent)
  sentiment = classifier.predict(prepared_text, False)
  print(sent, sentiment)

Я тебя люблю ['1']
Ненавиж всех вокруг. ['2']
Какой хороший день ['0']
Сегодня хорошая погода ['1']
Я радовался встрече с тобой ['1']
Мне нравится эта музыкальная композиция ['1']
В больнице была ужасная очередь ['2']
Сосед с верхнего этажа мешает спать ['2']
Маленькая девочка потерялась в торговом центре ['0']
Главный итог завершившихся Игр ХХХ Олимпиады в Лондоне – то чувство гордости за нашу страну, которое испытывали болельщики благодаря выступлениям российских олимпийцев», — считает Александр Жуков ['2']
На первый взгляд в квартире происходила борьба между мужчиной и женщиной. Во всяком случае, расположение тел об этом говорило. Но опытных сыщиков, которые прибыли на место преступления, насторожили некоторые детали, из чего они сделали вывод, что к убийству причастен кто-то посторонний. ['2']
Убил, виновен, понесет заслуженное наказание. Но почему журналисты делают такой акцент на компьютерной игре, в которой он насмотрелся сцен насилия, а не на дозе алкоголя, несовместимой с норм